### degrade_image_shift

In [1]:
import cv2
import numpy as np
import os
import json

In [2]:
# Папка с оригинальными изображениями
input_folder = "./content/DIV2K"



### filtered and  shifted

In [6]:
output_folder = "./content/filtered_shifted" 
meta_info_path = f"{output_folder}/.meta_info.json"  # Файл для метаинформации

os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения


# Функция ухудшения через смещение + снижение разрешения 
def degrade_image_shift(img, scale=5, shift_range=5):
    h, w = img.shape[:2]

    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)

    # 2 Генерация случайного сдвига
    shift_x = np.random.uniform(-shift_range, shift_range)  # Горизонтальный сдвиг
    shift_y = np.random.uniform(-shift_range, shift_range)  # Вертикальный сдвиг

    # 3 Матрица сдвига
    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])

    # 4 Применяем сдвиг
    shifted = cv2.warpAffine(blurred, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # 5 Уменьшаем разрешение (имитация ухудшения)
    small = cv2.resize(shifted, (w // scale, h // scale), interpolation=cv2.INTER_LINEAR)

    return small, shift_x, shift_y

# Количество ухудшенных версий для каждого изображения
num_versions = 10  
meta_info = {}


# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png"))]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    # Сохраняем метаинформацию о входном изображении
    meta_info[img_name] = {"original_size": img.shape[:2], "versions": []}

    # Генерируем несколько ухудшенных версий
    for i in range(num_versions):
        degraded_img, shift_x, shift_y = degrade_image_shift(img)

        # Сохраняем ухудшенное изображение
        save_path = f"{output_folder}/{img_name[:-4]}_v{i}.png"
        cv2.imwrite(save_path, degraded_img)

        # Сохраняем параметры преобразования в JSON
        meta_info[img_name]["versions"].append({
            "file": f"{img_name[:-4]}_v{i}.png",            
            "shift_x": shift_x,
            "shift_y": shift_y
        })
# Сохраняем метаинформацию в JSON
with open(meta_info_path, "w") as f:
    json.dump(meta_info, f, indent=4)

print("Good")

Good


### filtered, shifted and rotated

In [7]:
output_folder = "./content/filtered_shifted_rotated"
meta_info_path = f"{output_folder}/.meta_info.json"  # Файл для метаинформации

os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

# Функция ухудшения 
def degrade_image_transform(img, shift_range=5, rotation_range=10, scale=5):
    h, w = img.shape[:2]
    
    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)
    
    # 2 Генерируем случайные параметры
    shift_x = np.random.uniform(-shift_range, shift_range)  # Сдвиг по X
    shift_y = np.random.uniform(-shift_range, shift_range)  # Сдвиг по Y
    angle = np.random.uniform(-rotation_range, rotation_range)  # Поворот в градусах
    
    # 3 Генерируем координаты для выходного изображения 
    x_out, y_out = np.meshgrid(np.arange(w), np.arange(h))
    center = (w // 2, h // 2)
    
    # 4 Применяем преобразование подобия
    angle_rad = np.deg2rad(angle)  # Переводим градусы в радианы
    cos_fi, sin_fi = np.cos(angle_rad), np.sin(angle_rad)
    
    x_in = (cos_fi * (x_out - center[0]) + sin_fi * (y_out - center[1])) + center[0] + shift_x
    y_in = (-sin_fi * (x_out - center[0]) + cos_fi * (y_out - center[1])) + center[1] + shift_y
    
    # 5 Используем `cv2.remap()` для трансформации
    map_x, map_y = x_in.astype(np.float32), y_in.astype(np.float32)
    transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    
    # 6 Уменьшаем изображение
    degraded = cv2.resize(transformed, (w // scale, h // scale), interpolation=cv2.INTER_LINEAR)
    
    return degraded, shift_x, shift_y, angle

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith(".png")]

# Количество ухудшенных версий для каждого изображения
num_versions = 10 
meta_info = {}

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    # Сохраняем метаинформацию о входном изображении
    meta_info[img_name] = {"original_size": img.shape[:2], "versions": []}

    # Генерируем несколько ухудшенных версий
    for i in range(num_versions):
        degraded_img, shift_x, shift_y, angle = degrade_image_transform(img)

        # Сохраняем ухудшенное изображение
        save_path = f"{output_folder}/{img_name[:-4]}_v{i}.png"
        cv2.imwrite(save_path, degraded_img)
                        
        # Сохраняем параметры преобразования в JSON
        meta_info[img_name]["versions"].append({
            "file": f"{img_name[:-4]}_v{i}.png",            
            "shift_x": shift_x,
            "shift_y": shift_y,
            "angle": angle
        })

# Сохраняем метаинформацию в JSON
with open(meta_info_path, "w") as f:
    json.dump(meta_info, f, indent=4)

print("Good")


Good


### filtered, shifted, rotated and scaled

In [4]:
output_folder = "./content/filtered_shifted_rotated_scaled"
meta_info_path = f"{output_folder}/.meta_info.json"  # Файл для метаинформации

os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

# Функция ухудшения 
def degrade_image_transform(img, shift_range=5, rotation_range=10):
    h, w = img.shape[:2]
    
    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)
    
    # 2 Генерация случайных параметров
    shift_x = np.random.uniform(-shift_range, shift_range)  # Сдвиг по X
    shift_y = np.random.uniform(-shift_range, shift_range)  # Сдвиг по Y
    angle = np.random.uniform(-rotation_range, rotation_range)  # Поворот в градусах
    k = np.random.uniform(4.5, 5.5)  # Коэффициент масштаба
    
    # 3 Применяем сдвиг и поворот
    angle_rad = np.deg2rad(angle)  # Переводим градусы в радианы
    cos_fi, sin_fi = np.cos(angle_rad), np.sin(angle_rad)
    
    center = (w // 2, h // 2)  # Центр изображения
    x_out, y_out = np.meshgrid(np.arange(w), np.arange(h))  # Координаты в новом пространстве
    
    x_in = (cos_fi * (x_out - center[0]) + sin_fi * (y_out - center[1])) + center[0] + shift_x
    y_in = (-sin_fi * (x_out - center[0]) + cos_fi * (y_out - center[1])) + center[1] + shift_y
    
    # 4 Используем `cv2.remap()` для трансформации
    map_x, map_y = x_in.astype(np.float32), y_in.astype(np.float32)
    transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    
    # 5 Уменьшаем изображение в k раз
    small = cv2.resize(transformed, (int(w // k), int(h // k)), interpolation=cv2.INTER_LINEAR)
    
    # 6 Увеличиваем изображение обратно в исходный размер
    degraded = cv2.resize(small, (w, h), interpolation=cv2.INTER_LINEAR)
    
    # 7 Уменьшаем изображение в 5 раз
    degraded = cv2.resize(degraded, (w // 5, h // 5), interpolation=cv2.INTER_LINEAR)
    
    return degraded, shift_x, shift_y, angle, k

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith(".png")]

# Количество ухудшенных версий для каждого изображения
num_versions = 10
meta_info = {}

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)
    
    meta_info[img_name] = {"original_size": img.shape[:2], "versions": []}
    
    for i in range(num_versions):
        degraded_img, shift_x, shift_y, angle, k = degrade_image_transform(img)
        
        save_path = f"{output_folder}/{img_name[:-4]}_v{i}.png"
        cv2.imwrite(save_path, degraded_img)
        
        meta_info[img_name]["versions"].append({
            "file": f"{img_name[:-4]}_v{i}.png",
            "shift_x": shift_x,
            "shift_y": shift_y,
            "angle": angle,
            'scale': k
        })

with open(meta_info_path, "w") as f:
    json.dump(meta_info, f, indent=4)

print("Good")


Good
